<a href="https://colab.research.google.com/github/fspinola/test/blob/master/Copy_of_Task4_CNN_Binary_loss_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import resample

from skimage.io import imread


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
# Load raw training and testing data
train_raw_triplets = np.loadtxt('drive/My Drive/Colab Notebooks/train_triplets.txt', dtype='str')
testing_raw_triplets = np.loadtxt('drive/My Drive/Colab Notebooks/test_triplets.txt', dtype='str')
print ('done')

done


In [29]:
# Create new tripelts for validation: ACB
train_raw_triplets_new = train_raw_triplets.copy()
train_raw_triplets_new[:, 1] = train_raw_triplets[:, 2]
train_raw_triplets_new[:, 2] = train_raw_triplets[:, 1]
print('done')
print(train_raw_triplets)
print(train_raw_triplets_new)

done
[['02461' '03450' '02678']
 ['02299' '02499' '04987']
 ['04663' '01056' '03029']
 ...
 ['03255' '04844' '04334']
 ['02136' '04619' '00161']
 ['02509' '02552' '03406']]
[['02461' '02678' '03450']
 ['02299' '04987' '02499']
 ['04663' '03029' '01056']
 ...
 ['03255' '04334' '04844']
 ['02136' '00161' '04619']
 ['02509' '03406' '02552']]


In [32]:
# Add labels to validation data: ABC = 1 and ACB = 0
len_data = train_raw_triplets.shape[0]
labels_1 = np.ones(len_data)
labels_0 = 0*np.ones(len_data)
# split data into 2 : half is ABC and other half is ACB
labels = np.append(labels_1[0:math.floor(len_data/2)], labels_0[math.floor(len_data/2):], axis = 0)
print ('done')

done


In [33]:
# all validation data file names
train_raw_triplets_all = np.append(train_raw_triplets[0:math.floor(len_data/2)], train_raw_triplets_new[math.floor(len_data/2):], axis = 0)
print ('done')
print(train_raw_triplets_all)
print(labels)

done
[['02461' '03450' '02678']
 ['02299' '02499' '04987']
 ['04663' '01056' '03029']
 ...
 ['03255' '04334' '04844']
 ['02136' '00161' '04619']
 ['02509' '03406' '02552']]
[1. 1. 1. ... 0. 0. 0.]


In [35]:
# Split training IDs into training and validation
valid_size = 0.2 # percentage of training set to use as validation
ID_train, ID_val, train_labels, val_labels = train_test_split(train_raw_triplets_all, labels, test_size = valid_size) # train/val split
print((ID_train.shape))
print(ID_val.shape)

(47612, 3)
(11903, 3)


In [0]:
# Define data and network parameters
num_workers = 0
batch_size = 64
learning_rate = 0.01
epochs = 6

In [37]:
# Classes to turn data into pytorch dataset for dataloaders
## train data class
class trainID(torch.utils.data.Dataset):
    
    def __init__(self, X_ID, y_data):
        self.X_ID = X_ID
        self.X_ID_A = X_ID[:, 0]
        self.X_ID_B = X_ID[:, 1]
        self.X_ID_C = X_ID[:, 2]
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_ID_A[index], self.X_ID_B[index], self.X_ID_C[index], self.y_data[index]
        
    def __len__ (self):
        return self.X_ID.shape[0]
    

## test data class
class testID(torch.utils.data.Dataset):
    
    def __init__(self, X_ID):
        self.X_ID = X_ID
        self.X_ID_A = X_ID[:, 0]
        self.X_ID_B = X_ID[:, 1]
        self.X_ID_C = X_ID[:, 2]
        
    def __getitem__(self, index):
        return self.X_ID_A[index], self.X_ID_B[index], self.X_ID_C[index]
        
    def __len__ (self):
        return self.X_ID.shape[0]
    
print('done')

done


In [38]:
# Initialise dataloaders for training and validation sets
train_index = trainID(ID_train, train_labels)
print(type(train_index))
train_loader = DataLoader(dataset=train_index, batch_size=batch_size, shuffle=True, num_workers = num_workers)

val_index = trainID(ID_val_all, val_labels)
val_loader = DataLoader(dataset=val_index, batch_size=10, shuffle=True, num_workers = num_workers)

print('done')

<class '__main__.trainID'>
done


In [39]:
# Initialise dataloaders for testing set
print(testing_raw_triplets.shape)
test_index = testID(testing_raw_triplets)
test_loader = DataLoader(dataset=test_index, batch_size=36, shuffle=False, num_workers = num_workers)

print('done')

(59544, 3)
done


In [40]:
# Model Creation

model = torchvision.models.resnet18(pretrained=True)
print(model)

def resnet18_new(model):
    """
    Construct a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    modified_pretrained = nn.Sequential(*list(model.children())[:-1])
    for param in modified_pretrained.parameters():
        param.requires_grad = False
    
    modified_pretrained.add_module("embedding", EmbeddingNet(model))
    return modified_pretrained
    
class TripletNet(nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet
        #self.classifier = nn.Linear(2000, 1)

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_A = self.embeddingnet(a)

        # positive examples
        embedded_B = self.embeddingnet(p)

        # negative examples
        embedded_C = self.embeddingnet(n)
        
        # classifier
        #classi = embedded_A + embedded_B + embedded_C
        #classi = self.classifier(classi)
        
        return embedded_A, embedded_B, embedded_C#, classi


class EmbeddingNet(nn.Module):
    """EmbeddingNet using ResNet-18."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()

        # Everything except the last linear layer
        #self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs = resnet.fc.in_features
        self.fc1 = nn.Linear(num_ftrs, 1000)

    def forward(self, out):
        """Forward pass of EmbeddingNet."""
        #out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)

        return out

net = TripletNet(resnet18_new(model))
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
'''
# ZCA image whitening
def zca_whitening_matrix(X):
    """
    Function to compute ZCA whitening matrix (aka Mahalanobis whitening).
    INPUT:  X: [M x N] matrix.
        Rows: Variables
        Columns: Observations
    OUTPUT: ZCAMatrix: [M x M] matrix
    """
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True) # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U,S,V = np.linalg.svd(sigma)
        # U: [M x M] eigenvectors of sigma.
        # S: [M x 1] eigenvalues of sigma.
        # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 1e-5
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0/np.sqrt(S + epsilon)), U.T)) # [M x M]
    return ZCAMatrix
    '''

In [41]:
'''
# Load batch images from image IDs
def batch_images(sample_index, image_IDs):
    transforms1 = [torchvision.transforms.ToPILImage(),torchvision.transforms.Grayscale(num_output_channels=1),
                  torchvision.transforms.Resize((224,224))] # transforms to apply 
    transforms2 = [torchvision.transforms.ToTensor()]
    comp_transforms1 = torchvision.transforms.Compose(transforms1)
    comp_transforms2 = torchvision.transforms.Compose(transforms2)
    batch_img = []
    batch_img = torch.tensor(batch_img, dtype = torch.double)
    for i in sample_index:
        sample = []
        sample = torch.tensor(sample, dtype = torch.double)
        for j in range(image_IDs.shape[1]):
            img_path = 'food/food/' + image_IDs[i, j] +'.jpg'
            image = imread(img_path)
            image = comp_transforms1(image)
            image = np.array(image).reshape(224,224)
            ZCAMatrix = zca_whitening_matrix(image) # get ZCAMatrix
            image = np.dot(ZCAMatrix, image) # project X onto the ZCAMatrix
            image = torch.tensor(image, dtype = torch.double).reshape(1, 224, 224)/255
            #image = (comp_transforms2(image).reshape(1, 224, 224)).torch.double
            sample = torch.cat((sample, image), dim = 0)
        batch_img = torch.cat((batch_img, sample), dim = 0)
    batch_img = batch_img.reshape(-1, 3, 224, 224)
    return batch_img

print('done')
'''
# Load batch images from image IDs
def batch_images(image_ID):
    transforms = [torchvision.transforms.ToPILImage(),torchvision.transforms.Resize((150,224)), 
                  torchvision.transforms.ToTensor()] # transforms to apply 
    comp_transforms = torchvision.transforms.Compose(transforms)
    batch_img = []
    batch_img = torch.tensor(batch_img)
    for i in range(len(image_ID)):
        img_path = 'drive/My Drive/Colab Notebooks/food/food/' + image_ID[i] +'.jpg'
        image = imread(img_path)
        image = comp_transforms(image)
        #plt.figure()
        #plt.imshow(image.permute(1, 2, 0))
        batch_img = torch.cat((batch_img, image), dim = 0)
    batch_img = batch_img.reshape(-1, 3, 150, 224)
    return batch_img

print('done')

done


In [0]:
def pred_label(A, B, C):
    d_ab = torch.norm(A-B, p=2, dim=1)
    d_ac = torch.norm(A-C, p=2, dim=1)
    diff = (d_ac-d_ab).detach().cpu().numpy()
    y_pred = np.ceil(diff.clip(0, 1))
    y_pred = torch.tensor(y_pred, requires_grad=True).cuda()
    return y_pred

In [44]:
y_pred = torch.tensor([4.0, 4.0, 4.0], requires_grad=True).cuda()
print(y_pred)
y_true = np.array([4.0, 4.0, 4.0]).cuda()
print(y_true)
accuracy = accuracy_score(y_true, y_pred.detach().numpy())

tensor([4., 4., 4.], device='cuda:0', grad_fn=<CopyBackwards>)


AttributeError: ignored

In [47]:
model = net
if torch.cuda.is_available():
  model = model.cuda()
  print('cuda true')

# Specify loss function and optimiser
#criterion = nn.TripletMarginLoss(margin=1.0, p=2.0, reduce=None, reduction='mean')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam((model.embeddingnet.embedding.parameters()) , lr=learning_rate, weight_decay=0.01)
print('done')

cuda true
done


In [48]:
for A, B, C, y in train_loader:
    #print(A)
    print(batch_images(A))
    break

tensor([[[[0.5059, 0.5961, 0.8941,  ..., 0.1882, 0.2039, 0.2275],
          [0.5922, 0.8196, 0.8941,  ..., 0.1804, 0.1882, 0.1765],
          [0.7961, 0.9059, 0.8431,  ..., 0.1373, 0.1451, 0.1373],
          ...,
          [0.6549, 0.6353, 0.6196,  ..., 0.1529, 0.1882, 0.1490],
          [0.5843, 0.6039, 0.6549,  ..., 0.1765, 0.1725, 0.1765],
          [0.6471, 0.6235, 0.6275,  ..., 0.1961, 0.1882, 0.1843]],

         [[0.3647, 0.5137, 0.8784,  ..., 0.1882, 0.2078, 0.2196],
          [0.4667, 0.7490, 0.8941,  ..., 0.1961, 0.2118, 0.2000],
          [0.7020, 0.8667, 0.8627,  ..., 0.1765, 0.2000, 0.1922],
          ...,
          [0.3373, 0.3569, 0.3569,  ..., 0.1725, 0.2275, 0.2078],
          [0.3608, 0.3333, 0.3333,  ..., 0.1922, 0.2078, 0.2000],
          [0.3725, 0.3294, 0.3098,  ..., 0.2118, 0.2196, 0.2157]],

         [[0.2471, 0.4118, 0.8196,  ..., 0.1804, 0.2078, 0.2157],
          [0.3922, 0.6902, 0.8588,  ..., 0.1843, 0.2078, 0.2078],
          [0.6902, 0.8549, 0.8627,  ..., 0

In [0]:
plt.close
# Train model
valid_loss_min = np.Inf # track change in validation loss
valid_accuracy_max = 0.0 # track change in validation accuracy

for epoch in range(1, epochs+1):
    
    # keep track of training and validation loss
    train_loss = 0.0
    train_accuracy = 0.0
    valid_loss = 0.0
    valid_accuracy = 0.0
    
    ###################
    # train the model #
    ###################
    print('TRAINING')
    print('wait...')
    model.train()
    for A, B, C, y_train_true in train_loader:
        # retrieve batch of training images: tensor 25x3x150x224
        X_train_A = batch_images(A)
        X_train_B = batch_images(B)
        X_train_C = batch_images(C)

        if torch.cuda.is_available():
          X_train_A = X_train_A.cuda()
          X_train_B = X_train_B.cuda()
          X_train_C = X_train_C.cuda()
          y_train_true = y_train_true.cuda()

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        embedded_A, embedded_B, embedded_C = model(X_train_A, X_train_B, X_train_C)
        
        # calculate the batch loss
        y_pred = pred_label(embedded_A, embedded_B, embedded_C)
        loss = criterion(y_pred, y_train_true)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update training loss
        train_loss += loss.item()
        train_accuracy += accuracy_score(y_train_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy())
        
    # calculate average loss
    train_loss = train_loss/len(train_loader)
    train_accuracy = train_accuracy/len(train_loader)
    print('final acc')
    print(train_accuracy)
    
        
    ######################    
    # validate the model #
    ######################
    print('VALIDATION')
    print('wait...')
    model.eval()
    with torch.no_grad():
        for A, B, C, y_val_true in val_loader:
            # retrieve batch of training images: tensor 10x3x150x224
            X_val_A = batch_images(A)
            X_val_B = batch_images(B)
            X_val_C = batch_images(C)

            if torch.cuda.is_available():
              X_val_A=X_val_A.cuda()
              X_val_B=X_val_B.cuda()
              X_val_C=X_val_C.cuda()
              y_val_true = y_val_true.cuda()

            # forward pass: compute predicted outputs by passing inputs to the model
            embedded_A, embedded_B, embedded_C = model(X_val_A, X_val_B, X_val_C)

            # calculate the batch loss
            y_pred = pred_label(embedded_A, embedded_B, embedded_C)
            loss = criterion(y_pred, y_val_true)
            
            # update average validation loss 
            valid_loss += loss.item()
            # update training accuracy
            valid_accuracy += accuracy_score(y_val_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy())
            
    
    valid_loss = valid_loss/len(val_loader)
    valid_accuracy = valid_accuracy/len(val_loader)
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Epoch: {} \tTraining Acc: {:.6f} \tValidation Acc: {:.6f}'.format(
        epoch, train_accuracy, valid_accuracy))
    
    # save model if validation loss has decreased
    if valid_accuracy >= valid_accuracy_max:
        print('Validation acc increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_accuracy_max,
        valid_accuracy))
        torch.save(model.state_dict(), 'drive/My Drive/Colab Notebooks/model_cifar_cuda.pt')
        valid_accuracy_max = valid_accuracy
        

TRAINING
wait...


In [0]:
# Load model with highest accuracy
model.load_state_dict(torch.load('drive/My Drive/Colab Notebooks/model_cifar_cuda.pt'))
model = model.cuda()

In [24]:
# Test model and save labels

y_pred_list = []
model.eval()
# iterate over test data
with torch.no_grad():
    for A, B, C in test_loader:
        # retrieve batch of training images: tensor 20x9x200x200
        # retrieve batch of training images: tensor 10x3x150x224
        X_test_A = batch_images(A)
        X_test_B = batch_images(B)
        X_test_C = batch_images(C)

        if torch.cuda.is_available():
              X_test_A=X_test_A.cuda()
              X_test_B=X_test_B.cuda()
              X_test_C=X_test_C.cuda()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        embedded_A, _, _ = model(X_test_A, X_test_A, X_test_A)
        embedded_B, _, _ = model(X_test_B, X_test_B, X_test_B)
        embedded_C, _, _ = model(X_test_C, X_test_C, X_test_C)

        # convert output probabilities to predicted class
        d_ab = torch.norm(embedded_A-embedded_B, p=2, dim=1)
        d_ac = torch.norm(embedded_A-embedded_C, p=2, dim=1)
        diff = (d_ac-d_ab).detach().cpu().numpy()
        y_test_pred = np.ceil(diff.clip(0, 1))
        y_pred_list.append(y_test_pred) 
        
print(y_pred_list) 

[array([1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
       1., 0.], dtype=float32), array([1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0.], dtype=float32), array([1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1.], dtype=float32), array([0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0.], dtype=float32), array([0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 1.], dtype=float32), array([1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0.,

In [25]:
output = pd.DataFrame(np.array([0]), columns=['Predicted Label'])
y_pred_list = (np.array(y_pred_list)).reshape(-1, 1)
print(y_pred_list)
for i in range(len(y_pred_list)):
    output.loc[i,'Predicted Label']=y_pred_list[i]
    
#Save the output (predicted labels for the test features)
output.to_csv('drive/My Drive/Colab Notebooks/prediction_cuda.csv', index=False, header=False)

[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [0]:
# Finalise training data with features and labels and save them
X_ = train_img.reshape(-1, 9, 200, 200)
torch.save(X_, 'X_.pt')
y_ = torch.from_numpy(labels)
torch.save(y_, 'y_.pt')

print('done')

In [0]:
# Finalise testing data with features and save it
X_test = test_img.reshape(-1, 9, 200, 200)
torch.save(X_test, 'X_test.pt')

print('done')

torch.Size([119030])


In [0]:
# Load training data
X_ = torch.load('X_.pt')
y_ = torch.load('y_.pt')

print('done')

In [0]:
# Load testing data
X_test = torch.load('X_test.pt')

print ('done')

In [0]:
# Define network architecture